<a href="https://colab.research.google.com/github/kashindra-mahato/NLP/blob/main/LLAMA_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLAMA index
- llama index is a simple, flexible interface between external data and llms.
- We have used llama index for indexing our raw documents(in txt format), and query the index.
- We are using gpt 3.5 turbo model of openai for indexing and querying.
- Two modes namely compact, and QA are used here to demonstrate the effectiveness of gpt with llama indexing.

In [ ]:
!pip install llama-index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [ ]:
from llama_index import (
    LLMPredictor,
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    MockLLMPredictor,
    MockEmbedding,
    ServiceContext,
    StorageContext,
    PromptHelper,
    load_index_from_storage,
    ResponseSynthesizer,
    QuestionAnswerPrompt
    )
from langchain import OpenAI
from llama_index.node_parser import SimpleNodeParser
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.indices.postprocessor import (
    SimilarityPostprocessor,
    AutoPrevNextNodePostprocessor,
    PrevNextNodePostprocessor
    )
from llama_index.storage.docstore import SimpleDocumentStore
from langchain.chat_models import ChatOpenAI

## Indexing
- We define the model used for indexing and quering in LLMPredictor.
- PromptHelper helps to fill in the prompt, split the text, and fill in context information. Here we have used PromptHelper to define the parameters of openai model.
- ServiceContext is used for configuration, such as LLMPredictor(for configuring the LLM), the PromptHelper(for configuring input size/chunk size), the BaseEmbedding (for configuring embedding model).
- For index document store we have two options, we can either use Simple index store(SimpleIndexStore) or Vector stores. Here we have used a vector store(GPTVectorStoreIndex).
- We also have two options to use while indexing, either by diving the documents into nodes manually and then indexing those nodes or directly indexing the document(as we have done below). Indexing the document directly will divide the documents into nodes automatically.
- Then we are using persist method of storage_context, to save the index. It will save the index in as index_store.json, vector_store.json and docstore.json.

In [ ]:
def get_index(data_directory):
  # llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))
  llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
  max_input_size = 4090
  num_output = 256
  max_chunk_overlap = 20
  chunk_size_limit=256
  prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
  documents = SimpleDirectoryReader(data_directory).load_data()
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
  index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
  index.storage_context.persist(persist_dir=data_directory+"")

## model selection
- text-davinci-003 is a text generation model, which takes a prompt as input and responses the text as instructed.
- similarly gpt-3.5-turbo is a chat model, which has role specified as system, user and assistant, we define behavior of chatbot under system, and provide examples of chatbot interaction under user and assistant.
- text-davinci-003 cost 10 times more than gpt-3.5-turbo, therefore our choice of model is gpt-3.5-turbo. It can also act as retrieval model similar to text-davinci-003.
- We need to import gpt-3.5-turbo from langchain.chat_models as ChatOpenAI


### Loading the index
- To load the index from storage we have to use same ServiceContext used while indexing.

In [ ]:
def load_index(indexed_dir_path):
  llm_predictor = LLMPredictor()
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
  storage_context = StorageContext.from_defaults(persist_dir=indexed_dir_path)
  index = load_index_from_storage(storage_context=storage_context, service_context=service_context)
  return index

In [ ]:
documents_path = ""

In [ ]:
get_index(documents_path)

## Token Count

- LlamaIndex offers token predictors to predict token usage of LLM and embedding calls.
- We are using MockLLMPredictor to predict the token usage during the index querying.
- We are using MockEmbedding in tandem with MockPredictor for token usage of embedding calls.

In [ ]:
def get_indexing_token_count(data_directory):
  llm_predictor = MockLLMPredictor(max_tokens=256)
  embed_model = MockEmbedding(embed_dim=1536)
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  documents = SimpleDirectoryReader(data_directory).load_data()
  index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
  return llm_predictor.last_token_usage

In [ ]:
indexing_count = get_indexing_token_count(documents_path)

In [ ]:
indexing_count

0

### token count for compact method
- the compact mode will "compact" the promt during each LLM call by stuffing as many chunks to stuff in one prompt. Then "create and refine"(which is the default mode) an answer by going through multiple prompts.


In [ ]:
indexed = load_index(documents_path+"")
def get_token_count_c(query):
  index = indexed
  llm_predictor = MockLLMPredictor(max_tokens=256)
  embed_model = MockEmbedding(embed_dim=1536)
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
  )
  response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]
  )
  query_engine = RetrieverQueryEngine.from_args(retriever=retriever, response_synthesizer=response_synthesizer, service_context=service_context, response_mode='compact')
  response = query_engine.query(query)
  return response, llm_predictor.last_token_usage

In [ ]:
response, count = get_token_count_c("")

In [ ]:
response.response

In [ ]:
count

1391

### Token count for QA method
- A QuestionAnswerPompt is defined to answer the user queries based on the documents provided, and limiting the answer to the predefined number of words.

In [ ]:
indexed = load_index(documents_path+"")
def get_token_count_qa(query):
  index = indexed
  llm_predictor = MockLLMPredictor(max_tokens=256)
  embed_model = MockEmbedding(embed_dim=1536)
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  QA_PROMPT_TMPL = (
      "We have provided context information below. \n"
      "---------------------\n"
      "{context_str}"
      "\n---------------------\n"
      "Given this information, please answer the question in no more than 50 words: {query_str}\n"
  )
  QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)
  retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
  )
  response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]
  )
  query_engine = RetrieverQueryEngine.from_args(retriever=retriever, response_synthesizer=response_synthesizer, service_context=service_context, text_qa_template=QA_PROMPT)
  response = query_engine.query(query)
  return response, llm_predictor.last_token_usage

In [ ]:
response, count = get_token_count_qa("")

In [ ]:
count

2086

### Retrievers, Response Synthesizer and Query Engine
- In both compact and QA methods, we have used a retriever, a response synthesizer and a query engine.
- A retriever class retrieves a set of Nodes from an index given a query. We can specify a value of similarity_top_k which is the number top nodes to retrieve.
- A response synthesizer class takes in a set of Nodes and synthesizes an answer given a query. We can specify node_postprocessors, which is a list of post processors that can further enhance the quality of response generated.
- A query engine class takes in a query and returns a response object. It make use of retriever and response synthesizer modules under the hood.


### get answer method for 'compact'

In [ ]:
indexed = load_index(documents_path+"")
def get_answer(query):
  index = indexed
  retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
  )
  response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]
  )
  query_engine = RetrieverQueryEngine.from_args(retriever=retriever, response_synthesizer=response_synthesizer, response_mode='compact')
  response = query_engine.query(query)
  return response

In [ ]:
response = get_answer("")

In [ ]:
response.response

'\nThe classes available are those related to studying abroad, such as visa compliance, choosing the best course based on preferences, qualification, career goals, and financial circumstances.'

### get answer method for 'QA'

In [ ]:
indexed = load_index(documents_path+"")
def get_answer_qa(query):
  index = indexed
  QA_PROMPT_TMPL = (
      "We have provided context information below. \n"
      "---------------------\n"
      "{context_str}"
      "\n---------------------\n"
      "Given this information, please answer the question in less than 50 words: {query_str}\n"
  )
  QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)
  retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
  )
  response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]
  )
  query_engine = RetrieverQueryEngine.from_args(retriever=retriever, response_synthesizer=response_synthesizer, text_qa_template=QA_PROMPT)
  response = query_engine.query(query)
  return response


In [ ]:
response = get_answer_qa("")

In [ ]:
response.response

'\nTest preparation classes available include online classes, classes with qualified teachers and mentors, and practice exams.'

## Test case
- We have a total of 544 queries in test case.
- We are getting the token count and response from both 'compact' and 'QA prompt' methods and appending the result to the existing csv file along with query time for each question.

In [ ]:

import pandas as pd

In [ ]:
query_df = pd.read_excel('')

In [ ]:
import csv
from datetime import datetime
def query(test_data, indexed):
  for i in range(len(test_data['Queries '])):
    query = test_data['Queries '][i]
    t1 = datetime.now()
    answer_compact = get_answer(query)
    t2 = datetime.now()
    time_compact = t2-t1
    _, token_compact = get_token_count_c(query)
    t3 = datetime.now()
    answer_qa = get_answer_qa(query)
    t4 = datetime.now()
    time_qa = t4-t3
    _, token_qa = get_token_count_qa(query)
    print("\nQuery: ", query)
    print("\nAnswer(compact): ", answer_compact)
    print("\nAnswer(QA): ", answer_qa)
    # Create the dictionary (=row)
    row = {'Query':query,'answer_compact':answer_compact,'token_compact':token_compact, 'answer_qa':answer_qa, 'token_qa':token_qa, 'time_compact':time_compact, 'time_qa': time_qa}
    # Open the CSV file in "append" mode
    with open('.csv', 'a', newline='') as f:
        # Create a dictionary writer with the dict keys as column fieldnames
        writer = csv.DictWriter(f, fieldnames=row.keys())
        # Append single row to CSV
        writer.writerow(row)

In [ ]:
indexed = load_index(documents_path+"")

In [ ]:
query(query_df, indexed)